In [ ]:
import numpy as np
def find_closest_points(m, n):
  """
  기준 점 집합 m의 각 점에 대해 가장 가까운 점 n을 찾습니다.

  Args:
    m: 기준 점 집합 (NumPy 배열, M x 3)
    n: 이동할 점 집합 (NumPy 배열, N x 3)

  Returns:
    가장 가까운 점의 인덱스 (NumPy 배열, M)
  """

  distances = np.linalg.norm(m[:, np.newaxis, :] - n[np.newaxis, :, :], axis=2)
  closest_indices = np.argmin(distances, axis=1)
  return closest_indices

def compute_transformation(m, n):
  """
  특이값 분해 (SVD)를 사용하여 두 점 집합 간의 최적 변환을 계산합니다.

  Args:
    m: 기준 점 집합 (NumPy 배열, M x 3)
    n: 이동할 점 집합 (NumPy 배열, M x 3) (최근접점 매칭된 상태)

  Returns:
    rotation: 회전 행렬 (NumPy 배열, 3 x 3)
    translation: 이동 벡터 (NumPy 배열, 3)
  """

  # 중심 좌표 일치
  m_centered = m - np.mean(m, axis=0)
  n_centered = n - np.mean(n, axis=0)

  # 공분산 행렬 계산
  h = m_centered.T @ n_centered

  # 특이값 분해 (SVD)
  u, s, v = np.linalg.svd(h)

  # 회전 행렬 계산
  rotation = v @ u.T

  # 반사 문제 처리
  if np.linalg.det(rotation) < 0:
    rotation = -rotation

  # 이동 벡터 계산
  translation = np.mean(m, axis=0) - rotation @ np.mean(n, axis=0)

  return rotation, translation

def transform_points(points, t):
  """
  변환 행렬을 사용하여 점 집합을 변환합니다.

  Args:
    points: 변환할 점 집합 (NumPy 배열, N x 3)
    t: 변환 행렬 (NumPy 배열, 4 x 4)

  Returns:
    변환된 점 집합 (NumPy 배열, N x 3)
  """

  homogeneous_points = np.concatenate([points, np.ones((points.shape[0], 1))], axis=1)
  transformed_points = (t @ homogeneous_points.T).T[:, :3]
  return transformed_points
import open3d as o3d

def pt_to_pt(pt1, pt2):
  pcd1 = o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pt1))
  pcd2= o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pt2))
  reg_p2p = o3d.pipelines.registration.registration_icp(  # source, target
		pcd1, pcd2, max_correspondence_distance=10,  # Adjust the distance threshold as needed
		estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
		criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))
  pcd1 = pcd1.transform(reg_p2p.transformation)
  return np.asarray(pcd1.points)


def rotate_x(vertices, angle_degrees):
  """
  vertices를 x축에 대해 angle_degrees만큼 회전합니다.

  Args:
    vertices: (M, 3) 크기의 NumPy 배열
    angle_degrees: 회전 각도 (도 단위)

  Returns:
    rotated_vertices: (M, 3) 크기의 NumPy 배열, 회전된 vertices
  """

  angle_rad = (angle_degrees)*0.3  # 라디안 단위로 변환
  rotation_matrix = np.array([
      [1, 0, 0],
      [0, np.cos(angle_rad), -np.sin(angle_rad)],
      [0, np.sin(angle_rad), np.cos(angle_rad)]
  ])

  rotated_vertices = vertices @ rotation_matrix  # 행렬 곱셈
  return rotated_vertices
  

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
import k3d
import pickle
import numpy as np
import glob
from tqdm import tqdm
%matplotlib inline


filelist = glob.glob("/root/LV/LV/Neurips_LV/LBC_age_tgt/*.pkl")
# print(filelist)

for file in tqdm(filelist[10:15]):
    id = file.split("/")[-1][:-4]
    # copypath = "/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/oasis2/out/OAS20177_MR1_68_0_0_0.nii.gz_L/2500_OAS20177_MR1_68_0_0_0.nii.gz_L_30.26579475402832.npy"
    # id = copypath.split("/")[-2]
    with open("/root/LV/LV/2502LV/cuttail_temp.pkl", "rb") as f:
        data = pickle.load(f)
    file= f"/root/LV/LV/Neurips_LV/LBC_age_tgt/{id}.pkl"
    with open(file, "rb") as f:
        pt_data = pickle.load(f)

    scaler = np.load(glob.glob(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/mid2_LBC/out/{id}/1000*.npy")[-1])
    datavert = np.load(glob.glob(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/optim_mid2_LBC/out/{id}/6000*.npy")[-1])

    lv_pt = pt_data["lv"]
    hippo_pt = pt_data["hippo"]
    vert= data['cuttail_vert']
    
    # move to origin
    hippo_pt[:,0] -= lv_pt[:,0].min()
    vert[:,0] = vert[:,0] - vert[:,0].min()
    lv_pt[:,0] -= lv_pt[:,0].min()
    
    hippo_pt[:,1] -= lv_pt[:,1].max()
    vert[:,1] = vert[:,1] - vert[:,1].max()
    lv_pt[:,1] -= lv_pt[:,1].max()
    
    tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    vert[:,2] = vert[:,2] - (vert[:,2].max()+vert[:,2].min())/2

    # scaling of template mesh
    vert *= scaler[0,0,:]
    print(scaler[0,0,:])
    # datavert = np.load(copypath)[0]
    # # ICP of target shape PCD
    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # tgt_vert = pt_to_pt(tgt_vert, vert)
    # lv_pt = tgt_vert[:lv_pt.shape[0]]
    # hippo_pt = tgt_vert[lv_pt.shape[0]:]
    
    # # move to origin

    # hippo_pt[:,0] -= lv_pt[:,0].min()
    # vert[:,0] = vert[:,0] - vert[:,0].min()
    # lv_pt[:,0] -= lv_pt[:,0].min()
    
    # hippo_pt[:,1] -= lv_pt[:,1].max()
    # vert[:,1] = vert[:,1] - vert[:,1].max()
    # lv_pt[:,1] -= lv_pt[:,1].max()
    
    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    # lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    # vert[:,2] = vert[:,2] - (vert[:,2].max()+vert[:,2].min())/2
    
    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # tgt_vert = pt_to_pt(tgt_vert, vert)
    # lv_pt = tgt_vert[:lv_pt.shape[0]]
    # hippo_pt = tgt_vert[lv_pt.shape[0]:]
    
    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()-tgt_vert[:,2].min())/2
    # lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()-tgt_vert[:,2].min())/2
    # vert[:,2] = vert[:,2] - (vert[:,2].max()-vert[:,2].min())/2
    # # # Save target shape
    # tgt_pts = {"lv": lv_pt, "hippo": hippo_pt, "vert": vert}
    # full_id = file.split("/")[-1]
    # with open(f"/root/LV/LV/2502LV/pcd_scaled/L/notail/{full_id}", "wb") as f:
    #     pickle.dump(tgt_pts, f)
    
    plot = k3d.plot()
    plot.display()
    plot += k3d.points(lv_pt, point_size=1, color = 0x3f6bc5)
    plot += k3d.points(hippo_pt, point_size=1, color = 0x00ff00)
    plot += k3d.mesh(datavert, data['lv'], opacity = 0.5, color = 0xffff00)
    

  0%|          | 0/5 [00:00<?, ?it/s]

[0.90871626 1.0055734  0.866455  ]


Output()

[0.9763216 1.0626857 0.878744 ]


Output()

...

Output()

 80%|████████  | 4/5 [00:00<00:00, 34.05it/s]

[0.9129101 1.099241  0.9848285]


Output()

100%|██████████| 5/5 [00:00<00:00, 36.61it/s]


In [7]:
import k3d
import pickle
import numpy as np
import glob
from tqdm import tqdm
%matplotlib inline


filelist = glob.glob("/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_*_MR1_notail_L_opt_data.pkl")
print(filelist)
for file in tqdm(filelist[5:10]):
    with open("/root/LV/LV/2502LV/cuttail_temp.pkl", "rb") as f:
        data = pickle.load(f)
    with open(file, "rb") as f:
        pt_data = pickle.load(f)
    id = file.split("/")[-1].split("_")[1]
    scaler = np.load(glob.glob(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/mid_TRS3/out/{id}/2000_*.npy")[-1])

    lv_pt = pt_data["lv"]
    hippo_pt = pt_data["hippo"]
    vert= pt_data['vert']

    # move to origin

    hippo_pt[:,0] -= lv_pt[:,0].min()
    vert[:,0] = vert[:,0] - vert[:,0].min()
    lv_pt[:,0] -= lv_pt[:,0].min()
    
    hippo_pt[:,1] -= lv_pt[:,1].max()
    vert[:,1] = vert[:,1] - vert[:,1].max()
    lv_pt[:,1] -= lv_pt[:,1].max()
    
    tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    vert[:,2] = vert[:,2] - (vert[:,2].max()+vert[:,2].min())/2

    plot = k3d.plot()
    plot.display()
    plot += k3d.points(lv_pt, point_size=1, color = 0x3f6bc5)
    plot += k3d.points(hippo_pt, point_size=1, color = 0x00ff00)
    plot += k3d.mesh(vert, data['lv'], opacity = 0.5, color = 0xffff00)
    
    # scaling of template mesh
    vert *= scaler[0,0,:]
    
    plot = k3d.plot()
    plot.display()
    plot += k3d.points(lv_pt, point_size=1, color = 0x3f6bc5)
    plot += k3d.points(hippo_pt, point_size=1, color = 0x00ff00)
    plot += k3d.mesh(vert, data['lv'], opacity = 0.5, color = 0xffff00)

    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()-tgt_vert[:,2].min())/2
    # lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()-tgt_vert[:,2].min())/2
    # vert[:,2] = vert[:,2] - (vert[:,2].max()-vert[:,2].min())/2
    # # # Save target shape
    # tgt_pts = {"lv": lv_pt, "hippo": hippo_pt, "vert": vert}
    # full_id = file.split("/")[-1]
    # with open(f"/root/LV/LV/2502LV/pcd_scaled/L/notail/{full_id}", "wb") as f:
    #     pickle.dump(tgt_pts, f)
    

['/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0303_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0426_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0216_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0455_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0428_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0292_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0212_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0365_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0332_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0169_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0203_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0139_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/pcd_scaled/L/notail/OAS1_0072_MR1_notail_L_

  0%|          | 0/5 [00:00<?, ?it/s]

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

100%|██████████| 5/5 [00:00<00:00, 25.32it/s]


In [ ]:
:
    with open("/root/LV/LV/2502LV/cuttail_temp.pkl", "rb") as f:
        data = pickle.load(f)
    with open(file, "rb") as f:
        pt_data = pickle.load(f)
    id = file.split("/")[-1].split("_")[1]
    scaler = np.load(glob.glob(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/mid_TRS/out/{id}/2000_*.npy")[-1])

    lv_pt = pt_data["lv"]
    hippo_pt = pt_data["hippo"]
    vert= data['cuttail_vert']

    # move to origin
    hippo_pt[:,0] -= lv_pt[:,0].min()
    vert[:,0] = vert[:,0] - vert[:,0].min()
    lv_pt[:,0] -= lv_pt[:,0].min()
    hippo_pt[:,1:3] = hippo_pt[:,1:3] - vert[:,1:3].mean()
    lv_pt[:,1:3] = lv_pt[:,1:3] - vert[:,1:3].mean()
    vert[:,1:3] = vert[:,1:3] - vert[:,1:3].mean()

    # scaling of template mesh
    vert *= scaler[0,0,:]
    
    # ICP of target shape PCD
    tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    tgt_vert = pt_to_pt(tgt_vert, vert)
    lv_pt = tgt_vert[:lv_pt.shape[0]]
    hippo_pt = tgt_vert[lv_pt.shape[0]:]

    # Save target shape
    tgt_pts = {"lv": lv_pt, "hippo": hippo_pt, "vert": vert}
    full_id = file.split("/")[-1]
    with open(f"/root/LV/LV/2502LV/pcd_scaled/L/notail/{full_id}", "wb") as f:
        pickle.dump(tgt_pts, f)
    


    # plot = k3d.plot()
    # plot.display()
    # plot += k3d.points(lv_pt, point_size=1, color = 0x3f6bc5)
    # plot += k3d.points(hippo_pt, point_size=1, color = 0x00ff00)
    # plot += k3d.mesh(vert, data['lv'], opacity = 0.5, color = 0xffff00)
    

In [47]:
from pytorch3d.loss import mesh_laplacian_smoothing, mesh_normal_consistency
from pytorch3d.structures import Meshes 
import torch
with open("/root/LV/LV/2502LV/cuttail_temp.pkl", "rb") as f:
    data = pickle.load(f)
vertices=torch.from_numpy(data['cuttail_vert']).float()
lv_tri = torch.from_numpy(data['lv']).float()
hippo_tri = torch.from_numpy(data['hippo']).float()
# expand 0th dimensopm
vertices = vertices[None, :, :]
lv_tri = lv_tri[None, :, :]
hippo_tri = hippo_tri[None, :, :]
meshlv =Meshes(verts=list(vertices), faces=list(lv_tri))
meshhippo =Meshes(verts=list(vertices), faces=list(hippo_tri))
loss = mesh_laplacian_smoothing(meshlv)

print(mesh_normal_consistency(meshlv), mesh_normal_consistency(meshhippo))
print(mesh_laplacian_smoothing(meshlv,"uniform"), mesh_laplacian_smoothing(meshhippo,"uniform")) #cotcurv
print(mesh_laplacian_smoothing(meshlv,"cot"), mesh_laplacian_smoothing(meshhippo,"cot"))
print(mesh_laplacian_smoothing(meshlv,"cotcurv"), mesh_laplacian_smoothing(meshhippo,"cotcurv"))

tensor(0.1501) tensor(0.0364)
tensor(15.8591) tensor(116.7607)
tensor(15.8348) tensor(116.7513)
tensor(0.0724) tensor(0.0047)


In [48]:
a=(0.1501) /(0.0364)
b=(15.8591)/ (116.7607)
c=(15.8348) /(116.7513)
d=(0.0724) /(0.0047)
print(a,b,c,d)

4.123626373626374 0.13582566736924326 0.1356284683767975 15.404255319148938


In [68]:
plot = k3d.plot()
plot.display()

plot += k3d.mesh(data['cuttail_vert'], data['lv'], opacity_function = 0.1, color = 0xffff00)
plot += k3d.mesh(data['cuttail_vert'], data['hippo'], opacity_function = 0.1, color = 0xffff00)
    

Output()

/opt/conda/envs/pytorch3d/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/opt/conda/envs/pytorch3d/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


In [15]:
import k3d
import pickle
import numpy as np
import glob
%matplotlib inline



file = "/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0019_MR1_notail_L_opt_data.pkl"
filelist = glob.glob("/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_*_MR1_notail_L_opt_data.pkl")
print(filelist)
for file in filelist[:2]:
    with open("/root/LV/LV/2502LV/cuttail_temp.pkl", "rb") as f:
        data = pickle.load(f)
    print(data.keys())
    with open(file, "rb") as f:
        pt_data = pickle.load(f)
    id = file.split("/")[-1].split("_")[1]
    new_vert = np.load(glob.glob(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/mid_normal/out/0169/1000_*.npy")[-1])[0]
    # print(scaler, scaler.shape)
    lv_pt = pt_data["lv"]
    hippo_pt = pt_data["hippo"]
    vert= data['cuttail_vert']
    # plot += k3d.points(lv_pt, point_size=1, color = 0x3f6bc5)
    # plot += k3d.points(hippo_pt, point_size=1, color = 0x00ff00)
    # plot += k3d.mesh(vert, data['notail_lv'], opacity_function = 0.1, color = 0xffff00)

    plot2 = k3d.plot()
    plot2.display()

    # chg = np.asarray([lv_pt[:,0].min(),vert[:,1].mean(),vert[:,2].mean()])
    # lv_chg = np.asarray([0, vert[:,1].mean(),vert[:,2].mean()])
    # vert_chg = np.asarray([np.min(vert[:,0], 0, 0)])
    hippo_pt[:,0] -= lv_pt[:,0].min()
    vert[:,0] = vert[:,0] - vert[:,0].min()
    lv_pt[:,0] -= lv_pt[:,0].min()
    hippo_pt[:,1:3] = hippo_pt[:,1:3] - vert[:,1:3].mean()
    lv_pt[:,1:3] = lv_pt[:,1:3] - vert[:,1:3].mean()
    vert[:,1:3] = vert[:,1:3] - vert[:,1:3].mean()

    # vert -= vert_chg

    # vert *= scaler[0,0,:]
    
    vert= new_vert
    # # vert -= chg

    plot2 += k3d.points(lv_pt, point_size=1, color = 0x3f6bc5)
    plot2+= k3d.points(hippo_pt, point_size=1, color = 0x00ff00)

    plot2 += k3d.mesh(vert, data['lv'], opacity_function = 0.1, color = 0xffff00)


['/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0303_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0426_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0216_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0455_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0428_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0292_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0212_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0365_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0332_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0169_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0203_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0139_MR1_notail_L_opt_data.pkl', '/root/LV/LV/2502LV/normal_pcd/L/notail/OAS1_0072_MR1_notail_L_

Output()

dict_keys(['cuttail_vert', 'lv', 'hippo'])


Output()

In [9]:
import os
for fold in ["ADNI","AIBL","OASIS2","OASIS3"]:
    os.makedirs(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/mid2_{fold}")
    os.makedirs(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/mid2_{fold}/out")
    os.makedirs(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/mid2_{fold}/log")

In [10]:
import pickle
with open("/root/LV/LV/ADNI_17/002_S_0295_17_opt_data.pkl", "rb") as f:
    data = pickle.load(f)
print(data['0_002_S_0295_I13722_85_0_0_0.nii.gz'])

[[-11.13594486  17.06369403  -0.31386131]
 [-11.26185906  16.64061306   0.58343915]
 [-11.38777325  16.21753208   1.4807396 ]
 ...
 [ 13.60378912   3.54523153  -2.1018026 ]
 [ 13.80690357   4.86588995  -2.56505831]
 [ 13.68098937   4.44280898  -1.66775785]]


In [ ]:
# See longi LV


import k3d
import pickle
import numpy as np
import glob
from tqdm import tqdm
%matplotlib inline


filelist = glob.glob("/root/LV/LV/Neurips_LV/LBC_age_tgt_lv_optim/*.pkl")
# print(filelist)

for file in tqdm(filelist[:5]):
    id = file.split("/")[-1][:-4]
    # copypath = "/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/oasis2/out/OAS20177_MR1_68_0_0_0.nii.gz_L/2500_OAS20177_MR1_68_0_0_0.nii.gz_L_30.26579475402832.npy"
    # id = copypath.split("/")[-2]
    print(id)
    with open("/root/LV/LV/2502LV/cuttail_temp.pkl", "rb") as f:
        data = pickle.load(f)
    file= f"/root/LV/LV/Neurips_LV/LBC_age_tgt_lv_optim/LBC361232.pkl"
    with open(file, "rb") as f:
        data = pickle.load(f)
    print(data.keys())
    pt_data = data['LBC361232_2_76_1095_0_0.npy_L']
    zero_data = data['LBC361232_1_73_0_0_0.npy_L']

    datavert = np.load(glob.glob(f"/root/LV/lv-parametric-modelling/ipynb/MICCAI-LV/results/optim_mid2_LBC/out/LBC361232_1_73_0_0_0.npy_L/8000*.npy")[-1])

    lv_pt = pt_data["lv"]
    hippo_pt = pt_data["hippo"]
    vert= data['cuttail_vert']
    
    # move to origin
    hippo_pt[:,0] -= lv_pt[:,0].min()
    vert[:,0] = vert[:,0] - vert[:,0].min()
    lv_pt[:,0] -= lv_pt[:,0].min()
    
    hippo_pt[:,1] -= lv_pt[:,1].max()
    vert[:,1] = vert[:,1] - vert[:,1].max()
    lv_pt[:,1] -= lv_pt[:,1].max()
    
    tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    vert[:,2] = vert[:,2] - (vert[:,2].max()+vert[:,2].min())/2

    # scaling of template mesh
    vert *= scaler[0,0,:]
    print(scaler[0,0,:])
    # datavert = np.load(copypath)[0]
    # # ICP of target shape PCD
    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # tgt_vert = pt_to_pt(tgt_vert, vert)
    # lv_pt = tgt_vert[:lv_pt.shape[0]]
    # hippo_pt = tgt_vert[lv_pt.shape[0]:]
    
    # # move to origin

    # hippo_pt[:,0] -= lv_pt[:,0].min()
    # vert[:,0] = vert[:,0] - vert[:,0].min()
    # lv_pt[:,0] -= lv_pt[:,0].min()
    
    # hippo_pt[:,1] -= lv_pt[:,1].max()
    # vert[:,1] = vert[:,1] - vert[:,1].max()
    # lv_pt[:,1] -= lv_pt[:,1].max()
    
    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    # lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
    # vert[:,2] = vert[:,2] - (vert[:,2].max()+vert[:,2].min())/2
    
    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # tgt_vert = pt_to_pt(tgt_vert, vert)
    # lv_pt = tgt_vert[:lv_pt.shape[0]]
    # hippo_pt = tgt_vert[lv_pt.shape[0]:]
    
    # tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
    # hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()-tgt_vert[:,2].min())/2
    # lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()-tgt_vert[:,2].min())/2
    # vert[:,2] = vert[:,2] - (vert[:,2].max()-vert[:,2].min())/2
    # # # Save target shape
    # tgt_pts = {"lv": lv_pt, "hippo": hippo_pt, "vert": vert}
    # full_id = file.split("/")[-1]
    # with open(f"/root/LV/LV/2502LV/pcd_scaled/L/notail/{full_id}", "wb") as f:
    #     pickle.dump(tgt_pts, f)
    
    plot = k3d.plot()
    plot.display()
    plot += k3d.points(lv_pt, point_size=1, color = 0x3f6bc5)
    plot += k3d.points(hippo_pt, point_size=1, color = 0x00ff00)
    plot += k3d.mesh(datavert, data['lv'], opacity = 0.5, color = 0xffff00)
    
    

  0%|          | 0/5 [00:00<?, ?it/s]

LBC360697
dict_keys(['LBC361232_1_73_0_0_0.npy_L', 'LBC361232_1_73_0_0_0.npy_R', 'LBC361232_2_76_1095_0_0.npy_L', 'LBC361232_2_76_1095_0_0.npy_R'])


IndexError: list index out of range